# Experimentación métodos iterativos

In [ ]:
import numpy as np
import subprocess as sp
import matplotlib.pyplot as plt
import os

dir_datos = "../data/sistemas"
dir_resultados = "../data/resultados"
dir_tiempos = "../data/tiempos"
dir_figuras = "../data/figuras"

if not os.path.exists(dir_resultados):
    os.makedirs(dir_resultados)

if not os.path.exists(dir_datos):
    os.makedirs(dir_datos)

if not os.path.exists(dir_tiempos):
    os.makedirs(dir_tiempos)

if not os.path.exists(dir_figuras):
    os.makedirs(dir_figuras)

## Funciones auxiliares

In [ ]:
metodos = ["LU", "J", "JS", "GS", "GSS"]
nombre = ["Factorización LU", "Jacobi", "Jacobi Sum", "Gauss-Seidel", "Gauss-Seidel Sum"]
algoritmos = dict(zip(metodos, nombre))

In [ ]:
def correr_algoritmo(tam, metodo, iteraciones=0, tol=0):
    proceso = sp.run(["../src/iterativo", os.path.join(dir_datos, f"sistema_{tam}.txt"), metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)

def cargar_solucion(tam):    
    return np.genfromtxt(os.path.join(dir_datos, f"x_{tam}.txt"))

def guardar_errores_metodos(resultados, nombre, tam):
    np.savetxt(os.path.join(dir_resultados, f"{nombre}_{tam}.txt"), resultados, delimiter=" ")

def cargar_errores_metodos(nombre, tam):
    return np.genfromtxt(os.path.join(dir_resultados, f"{nombre}_{tam}.txt"))
    
def guardar_tiempos_metodos(tiempos, nombre, tam):
    np.savetxt(os.path.join(dir_tiempos, f"{nombre}_{tam}.txt"), tiempos, delimiter=" ")

def cargar_tiempos_metodos(nombre, tam):
    return np.genfromtxt(os.path.join(dir_tiempos, f"{nombre}_{tam}.txt"))

## Experimentación

In [ ]:
n = 2048
max_iter = 30000
tolerancia = 1e-10

x = cargar_solucion(n)
iters = range(1, max_iter)

### Jacobi

In [ ]:
errores_jacobi = np.array([])
for iter in iters:
    res = correr_algoritmo(n, "J", iter, tolerancia)
    errores_jacobi = np.append(errores_jacobi, np.linalg.norm(x - res))

guardar_errores_metodos(errores_jacobi, "jacobi", n)

In [ ]:
tiempos_jacobi = []
for i in range(2, int(np.log2(n)) + 1):
    t = %timeit -o correr_algoritmo(2 ** i, "J", max_iter, tolerancia)
    tiempos_jacobi.append((t.average, t.stdev))

guardar_tiempos_metodos(np.stack(tiempos_jacobi), "jacobi", n)

In [ ]:
tiempos_jacobi_sum = []
for i in range(2, int(np.log2(n)) + 1):
    t = %timeit -o correr_algoritmo(2 ** i, "JS", max_iter, tolerancia)
    tiempos_jacobi_sum.append((t.average, t.stdev))

guardar_tiempos_metodos(np.stack(tiempos_jacobi_sum), "jacobi_sum", n)

### Gauss-Seidel

In [ ]:
errores_gauss_seidel = np.array([])
for iter in iters:
    res = correr_algoritmo(n, "GS", iter, tolerancia)
    errores_gauss_seidel = np.append(errores_gauss_seidel, np.linalg.norm(x - res))

guardar_errores_metodos(errores_gauss_seidel, "gauss-seidel", n)

In [ ]:
tiempos_gauss_seidel = []
for i in range(2, int(np.log2(n)) + 1):
    t = %timeit -o correr_algoritmo(2 ** i, "GS", max_iter, tolerancia)
    tiempos_gauss_seidel.append((t.average, t.stdev))

guardar_tiempos_metodos(np.stack(tiempos_gauss_seidel), "gauss-seidel", n)

In [ ]:
tiempos_gauss_seidel_sum = []
for i in range(2, int(np.log2(n)) + 1):
    t = %timeit -o correr_algoritmo(2 ** i, "GSS", max_iter, tolerancia)
    tiempos_gauss_seidel_sum.append((t.average, t.stdev))

guardar_tiempos_metodos(np.stack(tiempos_gauss_seidel_sum), "gauss-seidel_sum", n)

### LU

In [ ]:
tiempos_lu = []
for i in range(2, int(np.log2(n)) + 1):
    t = %timeit -o correr_algoritmo(2 ** i, "LU")
    tiempos_lu.append((t.average, t.stdev))

guardar_tiempos_metodos(np.stack(tiempos_lu), "lu", n)

## Gráficos

### Error

In [ ]:
errores_gauss_seidel = cargar_errores_metodos("gauss-seidel", n)
errores_jacobi = cargar_errores_metodos("jacobi", n)

In [ ]:
plt.plot(iters, errores_gauss_seidel, "-", label="Gauss-Seidel")
plt.plot(iters, errores_jacobi, "-", label="Jacobi")
plt.xlabel("Cant. iteraciones")
plt.ylabel("Error")
plt.legend()
plt.grid(True)
plt.title(f"Error vs. para matriz de tamaño n = {n}")
plt.savefig(os.path.join(dir_figuras, f"error_vs_{n}.jpg"))

In [ ]:
plt.plot(iters, errores_gauss_seidel, "-", label="Gauss-Seidel")
plt.plot(iters, errores_jacobi, "-", label="Jacobi")
plt.xlabel("Cant. iteraciones")
plt.ylabel("Error")
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.title(f"Error vs. para matriz de tamaño n = {n} (log)")
plt.savefig(os.path.join(dir_figuras, f"error_vs_{n}_log.jpg"))

In [ ]:
plt.plot(iters[:15], errores_gauss_seidel[:15], "-", label="Gauss-Seidel")
plt.plot(iters[:15], errores_jacobi[:15], "-", label="Jacobi")
plt.xlabel("Cant. iteraciones")
plt.ylabel("Error")
plt.yscale("log")
plt.grid(True)
plt.legend()
plt.title(f"Error vs. para matriz de tamaño n = {n} (zoom - log)")
plt.savefig(os.path.join(dir_figuras, f"error_vs_{n}_zoom_log.jpg"))

In [ ]:
plt.plot(iters[800:1000], errores_jacobi[800:1000], "-")
plt.xlabel("Cant. iteraciones")
plt.ylabel("Error")
plt.yscale("log")
plt.grid(True)
plt.title(f"Error Jacobi para matriz de tamaño n = {n} (zoom - log)")
plt.savefig(os.path.join(dir_figuras, f"error_jacobi_{n}_zoom_log.jpg"))

### Tiempo de cómputo

In [ ]:
#tiempos_jacobi = cargar_tiempos_metodos("jacobi", n)
#tiempos_jacobi_sum = cargar_tiempos_metodos("jacobi_sum", n)
tiempos_gauss_seidel = cargar_tiempos_metodos("gauss-seidel", n)
tiempos_gauss_seidel_sum = cargar_tiempos_metodos("gauss-seidel_sum", n)
tiempos_lu = cargar_tiempos_metodos("lu", n)

In [ ]:
matrices = np.array([2 ** i for i in range(2, int(np.log2(n)) + 1)])

tj, dj = np.hsplit(tiempos_jacobi, 2)
plt.plot(matrices, tj, "s-", label="Jacobi")
plt.fill_between(matrices, tj.flatten() - dj.flatten(), tj.flatten() + dj.flatten(), alpha=0.4)

tjs, djs = np.hsplit(tiempos_jacobi_sum, 2)
plt.plot(matrices, tjs, "X-", label="Jacobi Sum")
plt.fill_between(matrices, tjs.flatten() - djs.flatten(), tjs.flatten() + djs.flatten(), alpha=0.4)

tgs, dgs = np.hsplit(tiempos_gauss_seidel, 2)
plt.plot(matrices, tgs, ">-", label="Gauss-Seidel")
plt.fill_between(matrices, tgs.flatten() - dgs.flatten(), tgs.flatten() + dgs.flatten(), alpha=0.4)

tgss, dgss = np.hsplit(tiempos_gauss_seidel_sum, 2)
plt.plot(matrices, tgss, "o-", label="Gauss-Seidel Sum")
plt.fill_between(matrices, tgss.flatten() - dgss.flatten(), tgss.flatten() + dgss.flatten(), alpha=0.4)

tl, dl = np.hsplit(tiempos_lu, 2)
plt.plot(matrices, tl, "d-", label="Factorización LU")
plt.fill_between(matrices, tl.flatten() - dl.flatten(), tl.flatten() + dl.flatten(), alpha=0.4)

plt.ylabel("Tiempo (s)")
plt.xlabel("Tamaño matriz")
plt.legend()
plt.grid(True)
plt.yscale("log")
plt.title("Tiempos de ejecución para todos los métodos (log)")
plt.savefig(os.path.join(dir_figuras, f"tiempos_vs_{n}_log.jpg"))

In [ ]:
matrices = np.array([2 ** i for i in range(2, int(np.log2(n)) + 1)])

tgs, dgs = np.hsplit(tiempos_gauss_seidel, 2)
plt.plot(matrices, tgs, ">-", label="Gauss-Seidel")
plt.fill_between(matrices, tgs.flatten() - dgs.flatten(), tgs.flatten() + dgs.flatten(), alpha=0.4)

tgss, dgss = np.hsplit(tiempos_gauss_seidel_sum, 2)
plt.plot(matrices, tgss, "o-", label="Gauss-Seidel Sum")
plt.fill_between(matrices, tgss.flatten() - dgss.flatten(), tgss.flatten() + dgss.flatten(), alpha=0.4)

tl, dl = np.hsplit(tiempos_lu, 2)
plt.plot(matrices, tl, "d-", label="Factorización LU")
plt.fill_between(matrices, tl.flatten() - dl.flatten(), tl.flatten() + dl.flatten(), alpha=0.4)

plt.ylabel("Tiempo (s)")
plt.xlabel("Tamaño matriz")
plt.legend()
plt.grid(True)
plt.yscale("log")
plt.title("Tiempos de ejecución métodos seleccionados (log)")
plt.savefig(os.path.join(dir_figuras, f"tiempos_selec_vs_{n}_log.jpg"))